# Naive Bayes Classifier

- 일단 조건부 독립에대해 설명(주사위 예시)
- 식으로 표현
- 코드
- 이산형 -> 연속형으로 가기 위해? class화한후 gauss변환

$x_1,...x_n$이 주어졌을 때 $y$의 class를 예측하는 문제를 조건부 확률로 접근<br>
$y$의 사건들의 집합을 $S=\{yes, no\}$로 가정하고 접근해보자.<br>
조건부확률의 정의에 의해 $$p(yes|x_1,...,x_n)=\frac{p(yes, x_1,...,x_n)}{p(x_1,...,x_n)}$$,
$$p(no|x_1,...,x_n)=\frac{p(no, x_1,...,x_n)}{p(x_1,...,x_n)}$$
로 나타낼 수 있다. 두 값들의 확률값을 비교해 확률이 높은 class를 택하면 되고, 두 식의 분모는 y의 class에 무관하므로 분자부분의 값들만 계산하면 된다.<br>


그런데 데이터의 크기가 작으면 $p(yes,x_1,...,x_n)$의 값은 0인 경우가 많다. 해당 사건들이 동시에 발생하는 경우가 등장하지 않기 때문이다. 나이브 베이즈는 **Naive한 가정(각 $x_i$는 나머지 $x_j$들에 대해 y가 주어진 조건하에 서로 독립이다 - 조건부 독립)** 을 이용해 이러한 문제를 해결할 수 있다.

$$p(yes,x_1,...,x_n)=p(x_1,...,x_n|yes)\times p(yes)$$<br>
$$=\frac{p(yes,x_1)}{p(yes)}\times \frac{p(yes,x_1,x_2)}{p(yes,x_1)}\times \cdots \times \frac{p(yes,x_1,...,x_n)}{p(yes,x_1,...,x_{n-1})}\times p(yes)$$로 나타낼 수 있고,

$\frac{p(yes,x_1,...,x_k)}{p(yes,x_1,...,x_{k-1})}=p(x_k|yes,x_1,...,x_{k-1})=p(x_k|yes)$(by 조건부 독립 가정)<br>
따라서 최종적으로 yes, no의 조건부 확률은 다음과 같이 나타낼 수 있다.<br>
$$p(yes|x_1,...,x_n) \propto p(x_1,...,x_n|yes)\times p(yes)$$
$$=p(yes)\times \frac{p(yes,x_1)}{p(yes)}\times \cdots \times \frac{p(yes,x_n)}{p(yes)}=p(yes)\prod^n_{i=1}p(x_i|yes)$$,
$$p(no|x_1,...,x_n) \propto p(no)\prod^n_{i=1}p(x_i|yes)$$

y의 class를 $C_k$로 확장하면 다음과 같이 나타낼 수 있다.
$$p(C_k|x_1,...,x_n) \propto p(C_k)\prod^n_{i=1}p(x_i|C_k)$$

위와같이 변환하면 $p(C_k|x1,...,x_n)$이 0으로 가는 case를 방지할 수 있다.<br>
예측의 경우 해당 확률을 최대화하는 k값으로 예측할 수 있으며, 확률의 경우 softmax를 적용해 구할 수 있을 것이다.<br>
softmax 참고 : $\hat{p}(C_k|x_1,...,x_n) = \frac{p(C_k)\prod^n_{i=1}p(x_i|C_k)}{\sum^n_{j=1}p(C_j)\prod^n_{i=1}p(x_i|C_j)}$

In [1]:
import pandas as pd
import numpy as np

## Make dataset

In [187]:
blood_types = ['A', 'B', 'AB', 'O']
glasses = [0, 1]

grade = ['A','B','C','D','F']

In [188]:
blood_types = ['A', 'A', 'B', 'B', 
               'B','AB', 'AB', 'O']
glasses = [0, 1, 0, 0, 1, 1, 0, 1]
passes = [0, 1, 1, 1, 0, 0, 1, 1]

In [189]:
df = pd.DataFrame({'blood_type': blood_types, 
                   'glasses': glasses,
                   'pass': passes
                  })

In [294]:
df

,blood_type,glasses,pass
0,A,0,0
1,A,1,1
2,B,0,1
3,B,0,1
4,B,1,0
5,AB,1,0
6,AB,0,1
7,O,1,1


In [291]:
X = df.iloc[:,:-1]

In [292]:
y = df.iloc[:,-1]

## Train

In [ ]:
def get_ratio(df, cx, cy):
    df = df.copy()
    df['count'] = 1  # count
    pivot = pd.pivot_table(df, index=cx, columns=cy, values='count', aggfunc=np.sum)
    pivot['sum'] = pivot.sum(axis=1)
    pivot = pivot.apply(lambda x:x[:-1] / x['sum'],axis=1)
    pivot = pivot.fillna(0)
    return pivot.to_dict()

In [296]:
def train(X, y):
    df = pd.concat([X,y], axis=1)
    col_x = df.columns[:-1]  # 열 이름들
    col_y = df.columns[-1]

    param = {}  # 학습된 파라미터들
    for cx in col_x:
        param[cx] = get_ratio(df, cx, col_y)

    y_ratio = {}  # 조건부 확률이 아닌 그냥 p(C_k)
    for y_val in y.unique():
        y_ratio[y_val] = np.mean(df[col_y] == y_val)
    param['y'] = y_ratio
    return param

In [297]:
train(X,y)

{'blood_type': {0: {'A': 0.5, 'AB': 0.5, 'B': 0.3333333333333333, 'O': 0.0},
  1: {'A': 0.5, 'AB': 0.5, 'B': 0.6666666666666666, 'O': 1.0}},
 'glasses': {0: {0: 0.25, 1: 0.5}, 1: {0: 0.75, 1: 0.5}},
 'y': {0: 0.375, 1: 0.625}}

blood_type을 예시로 설명하면 $P(y=0|blood\_type=A)=0.5,\ P(y=1|blood\_type=A)=0.5$ 라는 뜻이다.<br>
또 $P(y=0)=0.375,\ P(y=1)=0.625$ 이다.

blood_type이 B이고 pass=1인 경우는 2가지, blood_type이 B이고 pass=0인 경우는 한가지이므로<br>
$P(y=0|blood\_type=B)=0.666666$ 이 된다.

## Test

In [212]:
X_test = pd.DataFrame({'blood_type': ['A','B','O'], 'glasses': [0, 1, 0]})

In [299]:
def test(X_test):
    col_x = X_test.columns
    y_pred = []
    for i in range(len(X_test)):
        row = X_test.iloc[i]
        probs = []

        for y_val in y.unique():
            prob = param['y'][y_val]
            for i in range(len(col_x)):
                prob *= param[col_x[0]][y_val][row[0]]

            probs.append(prob)
        y_pred.append(probs)

    y_pred = pd.DataFrame(y_pred)
    y_pred['sum'] = y_pred.sum(axis=1)
    y_pred = y_pred.apply(lambda x:x[:-1]/x['sum'], axis=1)
    y_pred.columns = y.unique()
    return y_pred

In [301]:
X_test

,blood_type,glasses
0,A,0
1,B,1
2,O,0


In [303]:
test(X_test)  # 예측 결과

,0,1
0,0.375000,0.625000
1,0.130435,0.869565
2,0.000000,1.000000


## Result

In [302]:
df

,blood_type,glasses,pass
0,A,0,0
1,A,1,1
2,B,0,1
3,B,0,1
4,B,1,0
5,AB,1,0
6,AB,0,1
7,O,1,1


frequentist의 관점에서 살펴보면 $P(y=0|blood\_type=A,glasses=0)=P(y=0|blood\_type=A,glasses=1)=0.5$ 이지만,<br>
Naive한 가정하에 bayesian 관점에서 확률은 각각 0.375, 0.625로 차이를 보임을 알 수 있다.<br>
**Frequentist vs Bayesian :** 만약 blood_type과 glasses가 조건부(given pass)독립이 아니라면 빈도주의관점에서 접근하는게 더 정확하다.<br>
하지만 주어진 데이터가 얼마없을 경우, X의 차원이 클 경우 **각각의 빈도당 표본이 적어** 베이지안 관점이 좀더 정확한 결과를 낼 수 있을 것이다.